In [5]:
import yaml
import os
import json
import gzip
import zlib
import datetime
from dateutil import parser
import multiprocessing
from subprocess import run, PIPE
import os
from pymongo import MongoClient
client = MongoClient()
db = client.libs

In [9]:
os.listdir('./libs')

['Libraries.io-open-data-1.0.0.zip',
 'Readme.md',
 'repository_dependencies-1.0.0-2017-06-15.csv',
 'tags-1.0.0-2017-06-15.csv',
 'repositories-1.0.0-2017-06-15.csv',
 'versions-1.0.0-2017-06-15.csv',
 'basilene.py',
 'projects-1.0.0-2017-06-15.csv',
 'projects_with_repository_fields-1.0.0-2017-06-15.csv',
 'dependencies-1.0.0-2017-06-15.csv',
 'License.txt']

In [23]:
f= open('./libs/dependencies-1.0.0-2017-06-15.csv')
reader = csv.reader(f, delimiter=',')    

In [42]:
next(reader)

['8765',
 'Dub',
 'anchovy',
 '203',
 '0.5.0',
 '24',
 'anchovy:graphics',
 'Dub',
 'runtime',
 '',
 '~master',
 '']

In [18]:
versions_heads

['ID',
 'Platform',
 'Project Name',
 'Project ID',
 'Number',
 'Published Timestamp',
 'Created Timestamp',
 'Updated Timestamp']

In [22]:
for file in os.listdir('./libs'):
    if '.csv' in file:
        with open('./libs/{0}'.format(file)) as g:
            reader = csv.reader(g, delimiter=',')    
            heads = next(reader)
            example = next(reader)
            print('{0}: {1} \n {2} \n\n'.format(file, heads, example))

repository_dependencies-1.0.0-2017-06-15.csv: ['ID', 'Host Type', 'Repository Name with Owner', 'Repository ID', 'Manifest Platform', 'Manifest Filepath', 'Git branch', 'Manifest kind', 'Optional', 'Dependency Project Name', 'Dependency Requirements', 'Dependency Kind', 'Dependency Project ID'] 
 ['48593218', 'GitHub', 'brianmhunt/knockout-modal', '1', 'Bower', 'bower.json', '', 'manifest', '', 'knockout', '~3.2.0', 'development', '15957'] 


tags-1.0.0-2017-06-15.csv: ['ID', 'Host Type', 'Repository Name with Owner', 'Repository ID', 'Tag Name', 'Tag git sha', 'Tag Published Timestamp', 'Tag Created Timestamp', 'Tag Updated Timestamp'] 
 ['1', 'GitHub', 'brianmhunt/knockout-modal', '1', '0.7.0', 'b6bf1152b1e6e757a09812c30b713dec21a3a752', '2014-09-16 19:08:29 UTC', '2015-03-06 22:59:14 UTC', '2015-03-06 22:59:14 UTC'] 


repositories-1.0.0-2017-06-15.csv: ['ID', 'Host Type', 'Name with Owner', 'Description', 'Fork', 'Created Timestamp', 'Updated Timestamp', 'Last pushed Timestamp', 'H

In this release you will find data about software distributed and/or crafted publicly on the Internet. You will find information about its development, its distribution and its relationship with other software included as a dependency. You will not find any information about the individuals who create and maintain these projects.

Further information and documentation on this data set can be found at https://libraries.io/data

For enquiries please contact data@libraries.io

This dataset contains seven files:

## Projects
A project is a piece of software available on any one of the 33 package managers supported by Libraries.io.

## Versions
A Libraries.io version is an immutable published version of a Project from a package manager. Not all package managers have a concept of publishing versions, often relying directly on tags/branches from a revision control tool.

## Tags
A tag is equivalent to a tag in a revision control system. Tags are sometimes used instead of Versions where a package manager does not use the concept of versions. Tags are often semantic version numbers.

## Dependencies
Dependencies describe the relationship between a project and the software it builds upon. Dependencies belong to Version. Each Version can have different sets of dependencies. Dependencies point at a specific Version or range of versions of other projects.

## Repositories
A Libraries.io repository represents a publicly accessible source code repository from either github.com, gitlab.com or bitbucket.org. Repositories are distinct from Projects, they are not distributed via a package manager and typically an application for end users rather than component to build upon.

## Repository dependencies
A repository dependency is a dependency upon a Version from a package manager has been specified in a manifest file, either as a manually added dependency committed by a user or listed as a generated dependency listed in a lockfile that has been automatically generated by a package manager and committed.

## Projects with related Repository fields
This is an alternative projects export that denormalizes a projects related source code repository inline to reduce the need to join between two data sets.

# License
This dataset is released under the Creative Commons Attribution-ShareAlike 4.0 International License.

This license provides the user with the freedom to use, adapt and redistribute this data. In return the user must publish any derivative work under a similarly open license, attributing Libraries.io as a data source. The full text of the license is included in the data.

# Access, Attribution and Citation
The dataset is available to download from Zenodo at https://zenodo.org/record/808273.

Please attribute Libraries.io as a data source by including the words ‘Includes data from Libraries.io’ and reference the Digital Object identifier: 10.5281/Zenodo.808273.

In [3]:
with open('./libs/{0}'.format('dependencies-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
        print(headers)
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[header[i]] = field
            print(e_dict)
            break
#             db.dependencies.insert_one(e_dict)


51239987


In [44]:
with open('./libs/{0}'.format('projects-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        i=0
        for a in reader:
            print(a)
            break
            i+=1
        print(i)

2215454


### DataSet size:

projects: 2215454
dependencies: 51239987

In [14]:
with open('./libs/{0}'.format('dependencies-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
#         db.dependencies.drop()
        thousand = 0
        t_list = list()
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[headers[i]] = field
            thousand += 1
            t_list.append(e_dict)
            if thousand % 1000 == 0:
                thousand=0
                
                db.dependencies.insert_many(t_list)
                t_list = list()
        
        db.dependencies.insert_many(t_list)



In [13]:
with open('./libs/{0}'.format('projects-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
        db.projects.drop()
        thousand = 0
        t_list = list()
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[headers[i]] = field
            thousand += 1
            t_list.append(e_dict)
            if thousand % 1000 == 0:
                thousand=0
                
                db.projects.insert_many(t_list)
                t_list = list()
        
        db.projects.insert_many(t_list)
print(db.projects.count())

2215453


### things to do next

1. decide how to split the data to users and items
2. do filtering of dependencies
3. implement the simple MF
4. implement the recall or AUC

In [20]:
db.dependencies.find_one()

{'Dependency Kind': 'runtime',
 'Dependency Name': 'a4g:client',
 'Dependency Platform': 'Dub',
 'Dependency Project ID': '',
 'Dependency Requirements': '*',
 'ID': '8737',
 'Optional Dependency': '',
 'Platform': 'Dub',
 'Project ID': '197',
 'Project Name': 'a4g',
 'Version ID': '2',
 'Version Number': '0.1.0',
 '_id': ObjectId('594c1c845b2d8b5fa99273fa')}

In [17]:
def data_iter(cursor, batch_size=10000):
    mylist = list()
    it = 0
    for item in cursor:
        it +=1
        mylist.append(item)
        if it%batch_size==0:
            yield mylist
            it=0
            mylist= list()

cursor = db.dependencies.find()
batch_size=10000
cursor.batch_size(batch_size=batch_size)
my_iter = data_iter(cursor, batch_size)

In [19]:
deps_proj = dict()
proj_num = dict()

In [22]:
for entry in cursor:
    deps_proj[entry['Project ID']] = deps_proj.get(entry['Project ID'],0) + 1
    proj_num[entry['Dependency Project ID']] = proj_num.get(entry['Dependency Project ID'], 0) + 1

The dimension of 
projects: 775580
Referenced projects: 365434

In [25]:
import numpy as np
import tensorflow as tf

In [30]:
np.average(list(proj_num.values()))

140.21679975043372

In [33]:
proj_num['']

1212910

In [41]:
db.dependencies.count()

51239986

In [42]:
import networkx
